In [1]:
import torch
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import InferenceClient,notebook_login
from tqdm import tqdm

In [2]:
with open('ml1m/content/dataset/genres.txt', 'r') as f:
    genre_all = f.readlines()
genres = [genre.strip() for genre in genre_all]

mapping = {}
for genre, i in enumerate(genres):
    mapping[genre] = i

mapping

{0: 'Crime',
 1: 'Thriller',
 2: 'Fantasy',
 3: 'Horror',
 4: 'Sci-Fi',
 5: 'Comedy',
 6: 'Documentary',
 7: 'Adventure',
 8: 'Film-Noir',
 9: 'Animation',
 10: 'Romance',
 11: 'Drama',
 12: 'Western',
 13: 'Musical',
 14: 'Action',
 15: 'Mystery',
 16: 'War',
 17: "Children's"}

In [3]:
users = pd.read_csv('ml1m/content/dataset/users.dat', sep='::',
                        engine='python',
                        names=['userid', 'gender', 'age', 'occupation', 'zip']).set_index('userid')
ratings = pd.read_csv('ml1m/content/dataset/ratings.dat', engine='python',
                        sep='::', names=['userid', 'movieid', 'rating', 'timestamp'])
movies_train = pd.read_csv('ml1m/content/dataset/movies_train.dat', engine='python',
                        sep='::', names=['movieid', 'title', 'genre'], encoding='ISO-8859-1', index_col=False).set_index('movieid')
movies_test = pd.read_csv('ml1m/content/dataset/movies_test.dat', engine='python',
                        sep='::', names=['movieid', 'title', 'genre'], encoding='ISO-8859-1', index_col=False).set_index('movieid')
movies_train['genre'] = movies_train.genre.str.split('|')
movies_train.index.name = 'ID'
movies_test['genre'] = movies_test.genre.str.split('|')
movies_test.index.name = 'ID'

users.age = users.age.astype('int')
users.gender = users.gender.astype('category')
users.occupation = users.occupation.astype('category')
ratings.movieid = ratings.movieid.astype('int')
ratings.userid = ratings.userid.astype('int')

In [4]:
def preprocess(df, path='ml1m/content/dataset/ml1m-images', genres=genres) -> pd.DataFrame:
    df['label'] = df.genre.apply(lambda x: [1 if genre in x else 0 for genre in genres])
    df.drop(columns=['genre'], inplace=True)
    df.rename(columns={'title': 'text'}, inplace=True)
    df = df.reset_index(drop=True)
    return df

In [5]:
trainset = preprocess(movies_train)
trainset['content'] = ""
testset = preprocess(movies_test)

In [6]:
client = InferenceClient(
    "mistralai/Mixtral-8x7B-Instruct-v0.1"
)

In [7]:
def format_prompt(message, history):
    prompt = "<s>"
    for user_prompt, bot_response in history:
        prompt += f"[INST] {user_prompt} [/INST]"
        prompt += f" {bot_response}</s> "
    prompt += f"[INST] {message} [/INST]"
    return prompt

In [8]:
def generate(prompt, history, temperature=0.9, max_new_tokens=256, top_p=0.95, repetition_penalty=1.0):
    temperature = float(temperature)
    if temperature < 1e-2:
        temperature = 1e-2
    top_p = float(top_p)

    generate_kwargs = dict(
        temperature=temperature,
        max_new_tokens=max_new_tokens,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=True,
        seed=42,
    )

    formatted_prompt = format_prompt(prompt, history)

    stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=True, details=True, return_full_text=False)
    output = ""

    for response in stream:
        output += response.token.text
        yield output
    return output

In [29]:
notebook_login()
history = []

In [31]:
_template = "summarize the movie: "
for i in range(299,len(trainset)):
    _in = trainset.text[i]
    _out = _template + _in
    try:
        _out = generate(_out, history)
    except Exception as e:
        print("Paused at movie " + str(i))
        break
    _content = []
    for token in _out:
        _content.append(token)
    trainset.content[i] = _content[-2]
    if i%60==0:
        print("Generated " + str(i) + "movies")

HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/mistralai/Mixtral-8x7B-Instruct-v0.1 (Request ID: 7Fu7OOLy3REMHrHlli6d4)

Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate

In [24]:
print(trainset.content[299])

 "Turbulence" is a 1997 action thriller film directed by Robert Butler. The movie follows the story of a hijacked plane that gets diverted due to bad weather, leading to a tense and thrilling situation on board.

The film starts with Ryan Weaver, a serial killer, being transported on a commercial plane. However, things go awry when the plane hits turbulence, and the prisoners manage to break free. In the ensuing chaos, the pilots are killed, and the plane is left without anyone to fly it.

Meanwhile, on the ground, Stacy, a flight attendant who managed to escape the plane during the hijacking, tries to alert the authorities about the situation on board. She teams up with a disgraced air traffic controller, Terry, to help her take control of the plane.

Back on the plane, Ryan takes charge, manipulating and killing some of the passengers in the process. However, he soon realizes that Stacy and Terry are working together to retake control of the aircraft.

In a tense and thrilling finale